In [9]:
import os
import json
from pathlib import Path
from time import time
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# filepath = Path(__file__).parent
filepath = Path(os.path.abspath(''))
print(filepath)

/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks


In [10]:
# task_name = 'Bunya-from-datasets' # config!
task_name = 'bunya_processed'
data_dir = filepath / '../data'
raw_data_dir = data_dir / 'raw'
quality_gto_dir = raw_data_dir / 'Bunya-from-datasets/Quality_GTOs'

output_dir = data_dir / task_name
os.makedirs(output_dir, exist_ok=True)

print(raw_data_dir)
print(quality_gto_dir)
print(output_dir)

/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/raw
/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/raw/Bunya-from-datasets/Quality_GTOs
/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/bunya_processed


# contig_quality

In [11]:
# contig_quality
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.contig_quality')))}")
ex_file = 'GCF_031497195.1'
cq = pd.read_csv(quality_gto_dir / f'{ex_file}.contig_quality', sep='\t')
cq

Total files: 1572


,Contig,Segment,Copy_Num,Len,Min_Len,Max_len,AMB_Bases,Frac_AMB,Exceptions
0,NC_086348.1,Large RNA Segment,1,6374,5764.0,10736.0,0,0,NaN
1,NC_086347.1,Medium RNA Segment,1,4310,NaN,NaN,0,0,NaN
2,NC_086346.1,Small RNA Segment,1,1863,1350.0,3435.0,0,0,NaN


# feature_quality

In [12]:
# feature_quality
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.feature_quality')))}")
ex_file = 'GCF_031497195.1'
fq = pd.read_csv(quality_gto_dir / f'{ex_file}.feature_quality', sep='\t')
fq

Total files: 1572


,ID,Function,Evaluated,Prot_Len,Min_Len,Max_Len,Copy_Num,Exp_Copy_Num,Exceptions
0,fig|426786.38.CDS.1,Nucleocapsid protein (N protein),1.0,245,205.0,540.0,1.0,1.0,NaN
1,fig|426786.38.CDS.2,Small Nonstructural Protein NSs (NSs Protein),NaN,226,NaN,NaN,NaN,NaN,NaN
2,fig|426786.38.CDS.3,Pre-glycoprotein polyprotein GP complex (GPC p...,NaN,1366,NaN,NaN,NaN,NaN,NaN
3,fig|426786.38.CDS.4,RNA-dependent RNA polymerase (L protein),1.0,2083,1719.0,3211.0,1.0,1.0,NaN
4,fig|426786.38.mat_peptide.1,Phenuiviridae mature nonstructural 78-kD protein,NaN,647,NaN,NaN,NaN,NaN,NaN
5,fig|426786.38.mat_peptide.2,Mature envelope glycoprotein Gn (Gn protein),NaN,512,NaN,NaN,NaN,NaN,NaN
6,fig|426786.38.mat_peptide.3,Mature envelope glycoprotein Gc (Gc protein),NaN,506,NaN,NaN,NaN,NaN,NaN


# qual.gto

In [13]:
# qual.gto
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.qual.gto')))}")
ex_file = 'GCF_031497195.1'
file_path = quality_gto_dir / f'{ex_file}.qual.gto'

# Load the GTP file
with open(file_path, 'r') as file:
    gto = json.load(file)

for k in sorted(gto.keys()):
    print(f'{k}: {type(gto[k])}')

Total files: 1572
analysis_events: <class 'list'>
close_genomes: <class 'list'>
contigs: <class 'list'>
domain: <class 'str'>
features: <class 'list'>
genetic_code: <class 'int'>
id: <class 'str'>
ncbi_taxonomy_id: <class 'int'>
quality: <class 'dict'>
scientific_name: <class 'str'>


In [14]:
# Explore 'features' in the GTO dict
print(len(gto['features'])) # that's the number rows in feature_quality file
for d in gto['features']:
    print(sorted(d.keys())) # print sorted keys

# create a list of sorted dicts
fea = []
for d in gto['features']:
    fea.append({k: d[k] for k in sorted(d.keys())})

7
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']


In [15]:
fea[0]

{'annotations': [['Add feature',
   'annotate_by_viral_pssm',
   1742556319.00236,
   '2B74E144-0647-11F0-A1B3-509768D45E13'],
  ['Set function to Nucleocapsid protein (N protein)',
   'annotate_by_viral_pssm',
   1742556319.00236,
   '2B74E144-0647-11F0-A1B3-509768D45E13']],
 'family_assignments': [['Phenuiviridae',
   'Phenuiviridae.N.3.pssm',
   'Nucleocapsid protein (N protein)',
   'annotate_by_viral_pssm']],
 'feature_creation_event': '2B74E144-0647-11F0-A1B3-509768D45E13',
 'function': 'Nucleocapsid protein (N protein)',
 'id': 'fig|426786.38.CDS.1',
 'location': [['NC_086346.1', '70', '+', '738']],
 'protein_translation': 'MTDYADIAIAFAGEPINNAEVMGWVNEFAYEGFNAQRIIQLVQEKGPQTWQTDVKMMIVLALTRGNKPSKMIEKMSAEGKKKASRLITIYGLKSGNPGRDDLTLSRIAAAFAGWTCQALATLHPYLPVTGAAMDAISPGYPRAMMHPSFAGLIDNSIPEAYLQVVVDAHALYLLQFSRVINRNMRGQPKSVVVSSFLQPMNAAIVSGFISNDRRRKMLMAFGIVDQNGKPTAAVESAAKAFMTAV*',
 'type': 'CDS'}

In [16]:
for i, item in enumerate(fea):
    print(f"{item['id']},  {item['type']},  {item['function']}")

fig|426786.38.CDS.1,  CDS,  Nucleocapsid protein (N protein)
fig|426786.38.CDS.2,  CDS,  Small Nonstructural Protein NSs (NSs Protein)
fig|426786.38.CDS.3,  CDS,  Pre-glycoprotein polyprotein GP complex (GPC protein)
fig|426786.38.CDS.4,  CDS,  RNA-dependent RNA polymerase (L protein)
fig|426786.38.mat_peptide.1,  mat_peptide,  Phenuiviridae mature nonstructural 78-kD protein
fig|426786.38.mat_peptide.2,  mat_peptide,  Mature envelope glycoprotein Gn (Gn protein)
fig|426786.38.mat_peptide.3,  mat_peptide,  Mature envelope glycoprotein Gc (Gc protein)


# Summary data

In [17]:
# def summarize_data_from_gto(gto_file_path):
#     """
#     Extract protein data and metadata from a GTO file and return it as a DataFrame.
#     """
#     # file_path = quality_gto_dir / 'GCF_031497195.1.qual.gto'

#     ## Load the GTO file
#     with open(gto_file_path, 'r') as file:
#         gto = json.load(file)

#     ## 'features' key in GTO
#     # Each item in 'features' is a dict with keys: 'id', 'type', 'function', etc.
#     # fea is a list of dicts, each representing a 'features' item
#     fea = []
#     for d in gto['features']:
#         fea.append({k: d[k] for k in sorted(d.keys())})

#     # Aggregate the 'features' items into a DataFrame
#     df = []
#     # features_columns contains available data items from 'features' key
#     features_columns = ['id', 'type', 'function', 'protein_translation']
#     for i, item in enumerate(fea):
#         df.append([item[f] if f in item else np.nan for f in features_columns])
#         # df.append([item[f] for f in features_columns])
#     df = pd.DataFrame(df, columns=features_columns)
#     df['length'] = df['protein_translation'].apply(lambda x: len(x) if isinstance(x, str) else 0)

#     ## General, GTO file level metadata
#     df.insert(0, 'quality', gto['quality']['genome_quality'], allow_duplicates=True)
#     df.insert(0, 'file', gto_file_path.name, allow_duplicates=True)
#     return df

# Protein (feature) data

In [18]:
def get_protein_data_from_gto(gto_file_path):
    """
    Extract protein data and metadata from a GTO file and return it as a DataFrame.
    """
    # file_path = quality_gto_dir / 'GCF_031497195.1.qual.gto'

    ## Load the GTO file
    with open(gto_file_path, 'r') as file:
        gto = json.load(file)

    ## 'features' key in GTO
    # Each item in 'features' is a dict with keys: 'id', 'type', 'function', etc.
    # fea is a list of dicts, each representing a 'features' item
    fea = []
    for d in gto['features']:
        fea.append({k: d[k] for k in sorted(d.keys())})

    # Aggregate the 'features' items into a DataFrame
    df = []
    # features_columns contains available data items from 'features' key
    features_columns = ['id', 'type', 'function', 'protein_translation']
    for i, item in enumerate(fea):
        df.append([item[f] if f in item else np.nan for f in features_columns])
        # df.append([item[f] for f in features_columns])
    df = pd.DataFrame(df, columns=features_columns)
    df['length'] = df['protein_translation'].apply(lambda x: len(x) if isinstance(x, str) else 0)

    ## General, GTO file level metadata
    df.insert(0, 'quality', gto['quality']['genome_quality'], allow_duplicates=True)
    df.insert(0, 'file', gto_file_path.name, allow_duplicates=True)
    return df

In [19]:
# ex_file = 'GCF_031497195.1'
ex_file = 'GCA_000851025.1'
df = get_protein_data_from_gto(gto_file_path=quality_gto_dir / f'{ex_file}.qual.gto')
df

,file,quality,id,type,function,protein_translation,length
0,GCA_000851025.1.qual.gto,Poor,fig|3052303.150.mat_peptide.1,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),GTFTWTLSDSSGVENPGGYCLTKWMILAAELKCFGNTAVAKCNVNH...,234
1,GCA_000851025.1.qual.gto,Poor,fig|3052303.150.CDS.1,CDS,RNA-dependent RNA polymerase (L protein),MDEIISELRELCLNYIEQDERLSRQKLNFLGQREPRMVLIEGLKLL...,2211
2,GCA_000851025.1.qual.gto,Poor,fig|3052303.150.CDS.2,CDS,Pre-glycoprotein polyprotein GP complex (GPC p...,MGQIVTMFEALPHIIDEVINIVIIVLIVITGIKAVYNFATCGIFAL...,499
3,GCA_000851025.1.qual.gto,Poor,fig|3052303.150.CDS.3,CDS,Nucleocapsid protein (N protein),MSLSKEVKSFQWTQALRRELQSFTSDVKAAVIKDATNLLNGLDFSE...,559
4,GCA_000851025.1.qual.gto,Poor,fig|3052303.150.RNA.1,RNA,Small segment stemloop region,NaN,0


In [20]:
# Aggregate protein data from all GTO files
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.qual.gto')))}")

protein_data_fname = 'protein_data.tsv'

if not (output_dir / protein_data_fname).exists():
    print(f'Creating {output_dir / protein_data_fname}')
    dfs = []    
    for i, fpath in enumerate(sorted(quality_gto_dir.glob('*.gto'))):
        df = get_protein_data_from_gto(gto_file_path=fpath)
        dfs.append(df)

    prot_df = pd.concat(dfs, axis=0).reset_index(drop=True)
    prot_df.to_csv(output_dir / protein_data_fname, sep='\t', index=False)

else:
    print(f'Loading {output_dir / protein_data_fname}')
    prot_df = pd.read_csv(output_dir / protein_data_fname, sep='\t')

print(prot_df.shape)
print(prot_df.columns.tolist())

Total files: 1572
Creating /nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/bunya_processed/protein_data.tsv
(8400, 7)
['file', 'quality', 'id', 'type', 'function', 'protein_translation', 'length']


In [22]:
# Save DataFrame with samples that don't have protein data
no_prot_df = prot_df[prot_df['protein_translation'].isna()]
print(no_prot_df.shape)
no_prot_df.to_csv(output_dir / 'missing_protein_data.tsv', sep='\t', index=False)

(184, 7)


In [23]:
# Check duplicates
dups = prot_df[prot_df.duplicated(subset=['protein_translation'], keep=False)].sort_values(['protein_translation'])
dups.to_csv(output_dir / 'protein_duplicates.tsv', sep='\t', index=False)
print(dups.shape)
dups[:12]

(6646, 7)


,file,quality,id,type,function,protein_translation,length
5890,GCF_000871205.1.qual.gto,Good,fig|70566.1857.mat_peptide.3,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ACIQEKEIETIDDAAMCIGLYQNITQPKEYNTFVKELSSTLSSHEI...,936
250,GCA_000871205.1.qual.gto,Good,fig|70566.1858.mat_peptide.3,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ACIQEKEIETIDDAAMCIGLYQNITQPKEYNTFVKELSSTLSSHEI...,936
6838,GCF_002925575.1.qual.gto,Poor,fig|1145238.48.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPILEPGWSDTAHGVGEIPMKTDLELDFSLPSSSSYSYRRKLTN...,478
1220,GCA_002925575.1.qual.gto,Poor,fig|1145238.49.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPILEPGWSDTAHGVGEIPMKTDLELDFSLPSSSSYSYRRKLTN...,478
839,GCA_002117715.1.qual.gto,Good,fig|3052486.14.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLAEVGWSDTAHGVGVIPLKTDLELDFALASSASYSYRRKLSN...,476
6475,GCF_002117715.1.qual.gto,Good,fig|3052486.13.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLAEVGWSDTAHGVGVIPLKTDLELDFALASSASYSYRRKLSN...,476
6683,GCF_002817355.1.qual.gto,Poor,fig|3052490.13.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLLEPGWSDTAHGVGDIPMKTDLELDFAIPSSSSYSYRRRLVN...,476
1050,GCA_002817355.1.qual.gto,Poor,fig|3052490.14.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLLEPGWSDTAHGVGDIPMKTDLELDFAIPSSSSYSYRRRLVN...,476
6740,GCF_002830985.1.qual.gto,Good,fig|3052499.526.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLMESGWSDTAHGVGIIPMKTDLELDFALASSSSYSYRRKLVN...,476
133,GCA_000854765.2.qual.gto,Poor,fig|3052499.527.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLMESGWSDTAHGVGIIPMKTDLELDFALASSSSYSYRRKLVN...,476


# DNA/RNA (contigs) data

In [24]:
def get_dna_data_from_gto(gto_file_path):
    """
    Extract dna/rna data and metadata from a GTO file and return it as a DataFrame.
    """
    # file_path = quality_gto_dir / 'GCF_031497195.1.qual.gto'

    ## Load the GTO file
    with open(gto_file_path, 'r') as file:
        gto = json.load(file)

    ## 'contigs' key in GTO
    # Each item in 'contigs' is a dict with keys: 'id', 'replicon_type', 'replicon_geometry', 'dna'
    # ctg is a list of dicts, each representing a 'contigs' item
    cng = []
    for d in gto['contigs']:
        cng.append({k: d[k] for k in sorted(d.keys())})

    # Aggregate the 'contigs' items into a DataFrame
    df = []
    # contigs_columns contains available data items from 'contigs' key
    contigs_columns = ['id', 'replicon_type', 'replicon_geometry', 'dna']
    for i, item in enumerate(cng):
        df.append([item[f] if f in item else np.nan for f in contigs_columns])
    df = pd.DataFrame(df, columns=contigs_columns)
    df['length'] = df['dna'].apply(lambda x: len(x) if isinstance(x, str) else 0)

    ## General, GTO file level metadata
    df.insert(0, 'quality', gto['quality']['genome_quality'], allow_duplicates=True)
    df.insert(0, 'file', gto_file_path.name, allow_duplicates=True)
    return df

In [25]:
# Aggregate dna data from all GTO files
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.qual.gto')))}")

dna_data_fname = 'dna_data.tsv'

if not (output_dir / dna_data_fname).exists():
    print(f'Creating {output_dir / dna_data_fname}')
    dfs = []    
    for i, fpath in enumerate(sorted(quality_gto_dir.glob('*.gto'))):
        df = get_dna_data_from_gto(gto_file_path=fpath)
        dfs.append(df)

    dna_df = pd.concat(dfs, axis=0).reset_index(drop=True)
    dna_df.to_csv(output_dir / dna_data_fname, sep='\t', index=False)

else:
    print(f'Loading {output_dir / dna_data_fname}')
    dna_df = pd.read_csv(output_dir / dna_data_fname, sep='\t')

print(dna_df.shape)
print(dna_df.columns.tolist())

Total files: 1572
Creating /nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/bunya_processed/dna_data.tsv
(4686, 7)
['file', 'quality', 'id', 'replicon_type', 'replicon_geometry', 'dna', 'length']


In [26]:
# Save DataFrame with samples that don't have protein data
no_dna_df = dna_df[dna_df['dna'].isna()]
print(no_dna_df.shape)
no_dna_df.to_csv(output_dir / 'missing_dna_data.tsv', sep='\t', index=False)

(0, 7)


In [28]:
# df = dna_df.drop(columns=['dna', 'length'])
# df.to_csv(output_dir / 'dna_data_no_seq.tsv', sep='\t', index=False)

### 1. Check quality control in DNA data

In [ ]:
# DNA Quality Control
def summarize_dna_qc(df, seq_col='dna'):
    """ Perform quality control on DNA sequences in a DataFrame. """
    df = df.copy()
    ambiguous_codes = set("NRYWSKMBDHV")  # IUPAC ambiguous nucleotide codes
    
    qc = []
    for idx, seq in df[seq_col].items():
        if not isinstance(seq, str) or len(seq) == 0:
            qc.append((0, 0.0, 0, 0.0))  # Handle empty or missing sequences
            continue
        
        total_length = len(seq)
        ambiguous_count = sum(1 for base in seq if base in ambiguous_codes)
        ambiguous_fraction = ambiguous_count / total_length
        gc_content = (seq.count('G') + seq.count('C')) / total_length
        
        qc.append((ambiguous_count, ambiguous_fraction, total_length, gc_content))
    
    columns = ['ambiguous_count', 'ambiguous_fraction', 'length', 'gc_content']
    qc_df = pd.DataFrame(qc, columns=columns).reset_index(drop=True)
    df_combined = pd.concat([df, qc_df], axis=1)
    # return df, qc_df, df_combined
    return df_combined

dna_qc = summarize_dna_qc(dna_df, seq_col='dna')

# There is only sample with ambiguous_fraction of 0.169765. The rest are less than 0.035.
# TODO! How should we filter based on QC??
display(dna_qc[:3])
print(dna_qc['ambiguous_fraction'].value_counts().sort_index(ascending=False))

,file,quality,id,replicon_type,replicon_geometry,dna,length,ambiguous_count,ambiguous_fraction,length,gc_content
0,GCA_000847345.1.qual.gto,Good,DQ375403.1,Large RNA Segment,linear,ACACAAAGGCGCCCAATCATGGATTCTATATTATCAAAACAGCTGG...,6404,0,0.0,6404,0.438944
1,GCA_000847345.1.qual.gto,Good,DQ380206.1,Medium RNA Segment,NaN,ACACAAAGACGGTGCATTAAATGTATGTTTTATTAACAATTCTAAT...,3885,0,0.0,3885,0.454569
2,GCA_000847345.1.qual.gto,Good,DQ380151.1,Small RNA Segment,linear,ACACAAAGACCCCCTAGTGCTTATCAAGTATATCATGGATTACTTT...,1690,0,0.0,1690,0.489349


ambiguous_fraction
0.169765       1
0.033234       2
0.028332       1
0.023320       1
0.019280       1
            ... 
0.000138       2
0.000113       1
0.000083       1
0.000082       2
0.000000    4457
Name: count, Length: 147, dtype: int64


### 2. Check naming consistency in DNA data

In [ ]:
# All segment IDs are unique (no duplicates).
print(f"Total ids:  {dna_df['id'].count()}")
print(f"Unique ids: {dna_df['id'].nunique()}")

# Most segments follow the Large/Medium/Small RNA Segment convention.
# However, some segments are labeled as "Segment One", "Segment Two", ...
# 311 entries have missing values
# TODO! 1) How can we standardize the "Segment ..." labels? 2) How can we handle the missing values?
display(dna_df['replicon_type'].value_counts(dropna=False))

Total ids:  4686
Unique ids: 4686


replicon_type
Large RNA Segment     1503
Small RNA Segment     1371
Medium RNA Segment    1283
NaN                    311
Segment One             61
Segment Two             59
Segment Four            53
Segment Three           45
Name: count, dtype: int64

### 3. Check for duplicates

In [118]:
# Check duplicates
# There are duplicates.
# TODO! How should we handle the duplicates?
# 1. Are these dups necessarily a problem??
dna_dups = dna_df[dna_df.duplicated(subset=['dna'], keep=False)].sort_values(['dna'])
dna_dups.to_csv(output_dir / 'dna_duplicates.tsv', sep='\t', index=False)
print(dna_dups.shape)
dna_dups[:12]

(3208, 9)


,file,quality,id,replicon_type,replicon_geometry,dna,length,assembly_prefix,virus_id
2172,GCA_018595275.1.qual.gto,Poor,MW039262.1,Large RNA Segment,linear,AAAAAAAAAAAAAAAAAAAAAAAAAAAGTAGTAAACACACTCATAT...,6855,GCA,NaN
4391,GCF_018595275.1.qual.gto,Poor,NC_078362.1,Large RNA Segment,linear,AAAAAAAAAAAAAAAAAAAAAAAAAAAGTAGTAAACACACTCATAT...,6855,GCF,NaN
2467,GCA_029888265.1.qual.gto,Good,OL774846.1,Large RNA Segment,linear,AAAACAAAAAGAGATGGATCAAGACGTAGATCAAATTCTAAATAGA...,7004,GCA,NaN
4647,GCF_029888265.1.qual.gto,Good,NC_079031.1,Large RNA Segment,linear,AAAACAAAAAGAGATGGATCAAGACGTAGATCAAATTCTAAATAGA...,7004,GCF,NaN
3710,GCF_002831345.1.qual.gto,Good,NC_038733.1,Medium RNA Segment,linear,AAAACAGTAGTGTACCGCTGTGTATAAATAACCTAAGTGATAAAAG...,4411,GCF,NaN
602,GCA_002831345.1.qual.gto,Good,KP792662.1,Medium RNA Segment,linear,AAAACAGTAGTGTACCGCTGTGTATAAATAACCTAAGTGATAAAAG...,4411,GCA,NaN
4524,GCF_029887365.1.qual.gto,Poor,NC_078877.1,Small RNA Segment,linear,AAAACTTAAAGTCCTTTAATAGTATGGCTGAAGCGATTGTTTTCAA...,910,GCF,NaN
2332,GCA_029887365.1.qual.gto,Poor,MH507156.1,Small RNA Segment,linear,AAAACTTAAAGTCCTTTAATAGTATGGCTGAAGCGATTGTTTTCAA...,910,GCA,NaN
4314,GCF_018594865.1.qual.gto,Good,NC_078254.1,Large RNA Segment,linear,AAAACTTTTTCTCAAAGTTATAAATCCCTTACTCCCCCCTAAACCT...,12189,GCF,NaN
2095,GCA_018594865.1.qual.gto,Good,KU343163.1,Large RNA Segment,linear,AAAACTTTTTCTCAAAGTTATAAATCCCTTACTCCCCCCTAAACCT...,12189,GCA,NaN


In [ ]:
"""
1. Count how often each duplicated sequence appears across different files
This helps understand if duplicates are common within the same virus or across different ones.

Case 1:  2 duplicates.
This might be technical (e.g., GCA vs. GCF versions of the same assembly for a given virus). Some GTO files may contain reference (GCF) and draft (GCA) assemblies of the same virus, leading to identical sequences in different files.
- If these duplicates exist only between GCA/GCF pairs, we can prefer the GCF version (since it is typically a higher-quality reference).
- If they are between two different GCA assemblies, we need to check if they belong to the same virus (see next step).

Case 2:  > 2 duplicates.
Potential reasons:
- Shared sequences across different virus strains (real biological conservation).
- Multiple sequencing projects capturing the same virus but submitted separately.
- Annotation errors where different assemblies accidentally include redundant sequences.

What should we do?
- Check if they belong to different viruses (extract and compare assembly IDs, see next section).
- If they belong to different viruses, they might be real conserved elements → keep them.
- If they belong to the same virus, keep only one high-quality version to avoid redundancy.

ap:
Most duplicates appear in only 2 files. What does it mean? And, what should we do with those?
Then, we have 3 sequences that appear in 3 files, and 3 sequences that appear in 5 files. What does it mean? And, what should we do with those?
num_unique_files	count
2	1592
3	3
5	3
"""

dna_dups = dna_df[dna_df.duplicated(subset=['dna'], keep=False)].sort_values(['dna'])
# dna_dups.groupby('dna').agg(file=('file', 'unique')).reset_index().explode('file')

dna_dups_count_files = dna_dups[['dna', 'file']].groupby('dna')['file'].nunique().reset_index()
dna_dups_count_files['file'].value_counts().reset_index()

,file,count
0,2,1592
1,3,3
2,5,3


In [ ]:
# Follow up on step 1
# TODO
pass

In [ ]:
"""
2. Compare if duplicates occur between GCA and GCF assemblies of the same virus
If they are simply different versions of the same assembly, we may prefer to keep only one.

We extract 'assembly_prefix' and 'assembly_id' from the file names.

Why Extract the Assembly ID?
- Identify if duplicates are between GCA/GCF of the same virus (same assembly_id).
- Check if duplicates occur in different viruses (different assembly_id → likely real conservation).
- Ensure that if we remove duplicates, we are keeping the best assembly.
"""

# Extract file prefixes (e.g., GCA_XXXXX and GCF_XXXXX)
# dna_df['assembly_prefix'] = dna_df['file'].str.extract(r'^(GCA|GCF)_\d+')
dna_df[['assembly_pefix', 'assembly_id']] = dna_df['file'].str.extract(r'^(GCA|GCF)_(\d+\.\d+)')
# dna_df[['file', 'assembly_prefix', 'assembly_id']].head()

# Find duplicates occurring across GCA/GCF pairs (i.e., same sequence in both GCA and GCF)
dna_dups = dna_df[dna_df.duplicated(subset=['dna'], keep=False)].sort_values(['dna'])
dna_dups = dna_dups[['file', 'assembly_prefix', 'dna']]
gca_gcf_dups = dna_dups.groupby(['dna', 'assembly_prefix'])['file'].nunique().unstack()

# Display cases where duplicates exist in both GCA and GCF
gca_gcf_dups.dropna().reset_index()

assembly_prefix,dna,GCA,GCF
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAGTAGTAAACACACTCATAT...,1.0,1.0
1,AAAACAAAAAGAGATGGATCAAGACGTAGATCAAATTCTAAATAGA...,1.0,1.0
2,AAAACAGTAGTGTACCGCTGTGTATAAATAACCTAAGTGATAAAAG...,1.0,1.0
3,AAAACTTAAAGTCCTTTAATAGTATGGCTGAAGCGATTGTTTTCAA...,1.0,1.0
4,AAAACTTTTTCTCAAAGTTATAAATCCCTTACTCCCCCCTAAACCT...,1.0,1.0
...,...,...,...
1556,TTTCCATACTACGTGTACGAAGTTTTATATAATAAATAATAAATTA...,1.0,1.0
1557,TTTCTGAAGAATCGATAAGACAAATCAAAACAAATTAACAATCTCA...,1.0,1.0
1558,TTTGAATACAACACACATAGACGGCTCTAATTTGAAAAACATAAAA...,1.0,1.0
1559,TTTGCACCGGGGATCCTAGGCAAAGTGGTTATCTATTTTGGGACTC...,1.0,1.0


In [167]:
dna_df

,file,quality,id,replicon_type,replicon_geometry,dna,length,assembly_prefix,virus_id,assembly_id,assembly_pefix
0,GCA_000847345.1.qual.gto,Good,DQ375403.1,Large RNA Segment,linear,ACACAAAGGCGCCCAATCATGGATTCTATATTATCAAAACAGCTGG...,6404,GCA,NaN,000847345.1,GCA
1,GCA_000847345.1.qual.gto,Good,DQ380206.1,Medium RNA Segment,NaN,ACACAAAGACGGTGCATTAAATGTATGTTTTATTAACAATTCTAAT...,3885,GCA,NaN,000847345.1,GCA
2,GCA_000847345.1.qual.gto,Good,DQ380151.1,Small RNA Segment,linear,ACACAAAGACCCCCTAGTGCTTATCAAGTATATCATGGATTACTTT...,1690,GCA,NaN,000847345.1,GCA
3,GCA_000847645.1.qual.gto,Good,AB009656.1,Large RNA Segment,linear,ACACAAAGTCCTGGACAACAAAAACAAAAAAACTCTTTCATCAATA...,9760,GCA,NaN,000847645.1,GCA
4,GCA_000847645.1.qual.gto,Good,AB010376.1,NaN,NaN,ACACAAAGTCCTGGGCAAAAGAAAAGCAAAATAACCAACTATAGAT...,4056,GCA,NaN,000847645.1,GCA
...,...,...,...,...,...,...,...,...,...,...,...
4681,GCF_029888565.1.qual.gto,Good,NC_079116.1,Medium RNA Segment,linear,AGTAGTGTACTACCAATTATACAAAAGTTTCATTCAAAATGTTGTT...,4351,GCF,NaN,029888565.1,GCF
4682,GCF_029888565.1.qual.gto,Good,NC_079118.1,Small RNA Segment,linear,AGTAGTGTACTCCACAAAAAACAAAACTATAAATCGTTAGAAACAA...,923,GCF,NaN,029888565.1,GCF
4683,GCF_031497195.1.qual.gto,Good,NC_086346.1,Small RNA Segment,linear,ACACAAAGTTACCTAAACTTTGTGTTTGAGTCCTGAAATACAAACA...,1863,GCF,NaN,031497195.1,GCF
4684,GCF_031497195.1.qual.gto,Good,NC_086347.1,Medium RNA Segment,NaN,CTTTGTGTTTGACGGTGTTAATGAATTCAAAAAACATTTTTTATCT...,4310,GCF,NaN,031497195.1,GCF


In [161]:
dna_dups

,file,assembly_prefix,dna
2172,GCA_018595275.1.qual.gto,GCA,AAAAAAAAAAAAAAAAAAAAAAAAAAAGTAGTAAACACACTCATAT...
4391,GCF_018595275.1.qual.gto,GCF,AAAAAAAAAAAAAAAAAAAAAAAAAAAGTAGTAAACACACTCATAT...
2467,GCA_029888265.1.qual.gto,GCA,AAAACAAAAAGAGATGGATCAAGACGTAGATCAAATTCTAAATAGA...
4647,GCF_029888265.1.qual.gto,GCF,AAAACAAAAAGAGATGGATCAAGACGTAGATCAAATTCTAAATAGA...
3710,GCF_002831345.1.qual.gto,GCF,AAAACAGTAGTGTACCGCTGTGTATAAATAACCTAAGTGATAAAAG...
...,...,...,...
355,GCA_001924595.1.qual.gto,GCA,TTTGAATACAACACACATAGACGGCTCTAATTTGAAAAACATAAAA...
215,GCA_000930735.1.qual.gto,GCA,TTTGCACCGGGGATCCTAGGCAAAGTGGTTATCTATTTTGGGACTC...
3333,GCF_000930735.1.qual.gto,GCF,TTTGCACCGGGGATCCTAGGCAAAGTGGTTATCTATTTTGGGACTC...
4324,GCF_018594925.1.qual.gto,GCF,TTTGCTTCATTATACTTGAGCAGTGTGTGACATCTGAGGCAATGTT...


In [158]:
# # Explore commands
# dna_dups.groupby(['dna', 'assembly_prefix'])['file'].nunique().reset_index()
# dna_dups.groupby(['dna', 'assembly_prefix'])['file'].nunique()

In [ ]:
"""
3. Check if duplicates occur across different viruses
This helps determine if segments are widely shared or just repeated within the same virus.
"""

# Extract virus ID from 'file' (assuming it's part of the filename)
dna_df['virus_id'] = dna_df['file'].str.extract(r'G[CA]F?_(\d+)\.qual')

# Count how many unique viruses each duplicate appears in
dup_virus_counts = dna_df[dna_df.duplicated(subset=['dna'], keep=False)] \
    .groupby('dna')['virus_id'].nunique().reset_index()

dup_virus_counts.rename(columns={'virus_id': 'num_unique_viruses'}, inplace=True)
dup_virus_counts['num_unique_viruses'].value_counts()

num_unique_viruses
0    1598
Name: count, dtype: int64

In [ ]:
"""
4. Identify the best sequence to keep
For each set of duplicates, prioritize sequences as follows:
- Prefer Good quality over Poor.
- Prefer GCF (reference assembly) over GCA.
- If both are the same, keep only one per virus to avoid redundancy.
"""

# Sort by quality and assembly type to prioritize what to keep
dna_df['quality_score'] = dna_df['quality'].map({'Good': 1, 'Poor': 0})
dna_df['assembly_score'] = dna_df['file'].str.startswith('GCF').astype(int)

# Rank duplicates within each sequence group
dna_df['dup_rank'] = dna_df.groupby('dna')[['quality_score', 'assembly_score']].rank(
    method='first', ascending=False).sum(axis=1)

# Mark sequences to keep (rank = 1 means best sequence)
dna_df['keep'] = dna_df['dup_rank'] == 1

# Show cases where we would remove some duplicates
dna_df[dna_df.duplicated(subset=['dna'], keep=False) & ~dna_df['keep']]

In [ ]:
display(df[:20])

In [ ]:
display(df['quality'].value_counts())

In [ ]:
df['type'].value_counts()

In [ ]:
df['function'].value_counts()